In [253]:
import numpy as np

# mne imports
import mne
from mne import io, read_epochs, concatenate_epochs
from mne.datasets import sample
from mne import Epochs, create_info, events_from_annotations
from mne.datasets import eegbci
from mne.decoding import CSP
from mne.time_frequency import AverageTFR

# EEGNet-specific imports
from EEGModels import EEGNet
from tensorflow.keras import utils as np_utils
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import backend as K

# PyRiemann imports
from pyriemann.estimation import XdawnCovariances
from pyriemann.tangentspace import TangentSpace
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
# tools for plotting confusion matrices
from matplotlib import pyplot as plt

In [254]:
cd epochs

[Errno 2] No such file or directory: 'epochs'
/Users/jang-wonjun/Documents/eeg/ml/epochs


In [255]:
ls

epoch0-epo.fif   epoch2-epo.fif   epoch5-epo.fif   epoch8-epo.fif
epoch1-epo.fif   epoch3-epo.fif   epoch6-epo.fif   epoch9-epo.fif
epoch10-epo.fif  epoch4-epo.fif   epoch7-epo.fif


In [256]:
n_epochs = 11
epoch_list = [f'epoch{i}-epo.fif' for i in range(n_epochs)]

In [257]:
epoch = concatenate_epochs([read_epochs(epoch_i) for epoch_i in epoch_list])
epoch

Reading /Users/jang-wonjun/Documents/eeg/ml/epochs/epoch0-epo.fif ...
    Found the data of interest:
        t =   -2000.00 ...    2000.00 ms
        0 CTF compensation matrices available
Not setting metadata
10 matching events found
No baseline correction applied
0 projection items activated
Reading /Users/jang-wonjun/Documents/eeg/ml/epochs/epoch1-epo.fif ...
    Found the data of interest:
        t =   -2000.00 ...    2000.00 ms
        0 CTF compensation matrices available
Not setting metadata
10 matching events found
No baseline correction applied
0 projection items activated
Reading /Users/jang-wonjun/Documents/eeg/ml/epochs/epoch2-epo.fif ...
    Found the data of interest:
        t =   -2000.00 ...    2000.00 ms
        0 CTF compensation matrices available
Not setting metadata
10 matching events found
No baseline correction applied
0 projection items activated
Reading /Users/jang-wonjun/Documents/eeg/ml/epochs/epoch3-epo.fif ...
    Found the data of interest:
        t =  

Number of events,110
Events,finger_movement: 110
Time range,-2.000 – 2.000 s
Baseline,off


In [272]:
fmin, fmax =2 ,8
#fmin,fmax=None,None
epoch_filter = epoch.copy().filter(fmin,fmax)

Setting up band-pass filter from 2 - 8 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 2.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 1.00 Hz)
- Upper passband edge: 8.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 9.00 Hz)
- Filter length: 213 samples (1.664 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 770 out of 770 | elapsed:    0.1s finished


In [273]:
w_tmin, w_tmax = 0, 0.5
on_event = epoch_filter.copy().crop(w_tmin,w_tmax)
off_event = epoch_filter.copy().crop(-2+w_tmin,-2+w_tmax)
        
X = np.concatenate([on_event.get_data(),off_event.get_data()],axis=0)*1000
y = [1 for i in range(X.shape[0]//2)]+[0 for i in range(X.shape[0]//2)]

In [274]:
X.shape # (trials,channels,samples)

(220, 7, 65)

In [275]:
kernels, chans, samples = 1, 7, 65

In [276]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2, random_state=42)


In [277]:
X_val,X_test,y_val,y_test = train_test_split(X_test, y_test,test_size=0.5, random_state=42)

In [278]:
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
y_val = np_utils.to_categorical(y_val)

In [279]:
model = EEGNet(nb_classes = 2, Chans = chans, Samples = samples, 
               dropoutRate = 0.5, kernLength = 32, F1 = 8, D = 2, F2 = 16, 
               dropoutType = 'Dropout')

model.compile(loss='categorical_crossentropy', optimizer='adam', 
              metrics = ['accuracy'])

In [280]:
numParams    = model.count_params()

In [281]:
checkpointer = ModelCheckpoint(filepath='/tmp/checkpoint.h5', verbose=1,
                               save_best_only=True)

In [282]:
class_weights = {0:1, 1:1}

In [283]:
fittedModel = model.fit(X_train, y_train, batch_size = 16, epochs = 300, 
                        verbose = 2, validation_data=(X_val, y_val),
                        callbacks=[checkpointer], class_weight = class_weights)

Epoch 1/300

Epoch 1: val_loss improved from inf to 0.69316, saving model to /tmp/checkpoint.h5
11/11 - 0s - loss: 0.7016 - accuracy: 0.4943 - val_loss: 0.6932 - val_accuracy: 0.5000 - 322ms/epoch - 29ms/step
Epoch 2/300

Epoch 2: val_loss did not improve from 0.69316
11/11 - 0s - loss: 0.6886 - accuracy: 0.5284 - val_loss: 0.6932 - val_accuracy: 0.5000 - 40ms/epoch - 4ms/step
Epoch 3/300

Epoch 3: val_loss did not improve from 0.69316
11/11 - 0s - loss: 0.7028 - accuracy: 0.4773 - val_loss: 0.6932 - val_accuracy: 0.5000 - 40ms/epoch - 4ms/step
Epoch 4/300

Epoch 4: val_loss did not improve from 0.69316
11/11 - 0s - loss: 0.6987 - accuracy: 0.5114 - val_loss: 0.6932 - val_accuracy: 0.5000 - 37ms/epoch - 3ms/step
Epoch 5/300

Epoch 5: val_loss did not improve from 0.69316
11/11 - 0s - loss: 0.6836 - accuracy: 0.5739 - val_loss: 0.6932 - val_accuracy: 0.5000 - 40ms/epoch - 4ms/step
Epoch 6/300

Epoch 6: val_loss did not improve from 0.69316
11/11 - 0s - loss: 0.6938 - accuracy: 0.5114 - 

11/11 - 0s - loss: 0.4742 - accuracy: 0.8011 - val_loss: 0.7669 - val_accuracy: 0.5000 - 39ms/epoch - 4ms/step
Epoch 48/300

Epoch 48: val_loss did not improve from 0.69313
11/11 - 0s - loss: 0.4542 - accuracy: 0.8239 - val_loss: 0.7995 - val_accuracy: 0.5000 - 40ms/epoch - 4ms/step
Epoch 49/300

Epoch 49: val_loss did not improve from 0.69313
11/11 - 0s - loss: 0.4344 - accuracy: 0.8352 - val_loss: 0.7951 - val_accuracy: 0.5000 - 39ms/epoch - 4ms/step
Epoch 50/300

Epoch 50: val_loss did not improve from 0.69313
11/11 - 0s - loss: 0.4364 - accuracy: 0.8693 - val_loss: 0.7939 - val_accuracy: 0.5000 - 37ms/epoch - 3ms/step
Epoch 51/300

Epoch 51: val_loss did not improve from 0.69313
11/11 - 0s - loss: 0.4099 - accuracy: 0.8636 - val_loss: 0.7807 - val_accuracy: 0.5000 - 39ms/epoch - 4ms/step
Epoch 52/300

Epoch 52: val_loss did not improve from 0.69313
11/11 - 0s - loss: 0.4459 - accuracy: 0.8239 - val_loss: 0.7734 - val_accuracy: 0.5000 - 38ms/epoch - 3ms/step
Epoch 53/300

Epoch 53: 

Epoch 90/300

Epoch 90: val_loss improved from 0.34660 to 0.34366, saving model to /tmp/checkpoint.h5
11/11 - 0s - loss: 0.3435 - accuracy: 0.8580 - val_loss: 0.3437 - val_accuracy: 1.0000 - 54ms/epoch - 5ms/step
Epoch 91/300

Epoch 91: val_loss did not improve from 0.34366
11/11 - 0s - loss: 0.3375 - accuracy: 0.8920 - val_loss: 0.3446 - val_accuracy: 1.0000 - 37ms/epoch - 3ms/step
Epoch 92/300

Epoch 92: val_loss improved from 0.34366 to 0.33785, saving model to /tmp/checkpoint.h5
11/11 - 0s - loss: 0.2805 - accuracy: 0.9261 - val_loss: 0.3379 - val_accuracy: 1.0000 - 53ms/epoch - 5ms/step
Epoch 93/300

Epoch 93: val_loss did not improve from 0.33785
11/11 - 0s - loss: 0.3451 - accuracy: 0.8864 - val_loss: 0.3385 - val_accuracy: 1.0000 - 39ms/epoch - 4ms/step
Epoch 94/300

Epoch 94: val_loss did not improve from 0.33785
11/11 - 0s - loss: 0.3259 - accuracy: 0.8807 - val_loss: 0.3413 - val_accuracy: 1.0000 - 38ms/epoch - 3ms/step
Epoch 95/300

Epoch 95: val_loss did not improve from 0

11/11 - 0s - loss: 0.2658 - accuracy: 0.9148 - val_loss: 0.3026 - val_accuracy: 0.8182 - 39ms/epoch - 4ms/step
Epoch 134/300

Epoch 134: val_loss did not improve from 0.25815
11/11 - 0s - loss: 0.2630 - accuracy: 0.9261 - val_loss: 0.2958 - val_accuracy: 0.8182 - 43ms/epoch - 4ms/step
Epoch 135/300

Epoch 135: val_loss did not improve from 0.25815
11/11 - 0s - loss: 0.2513 - accuracy: 0.9432 - val_loss: 0.3130 - val_accuracy: 0.7727 - 37ms/epoch - 3ms/step
Epoch 136/300

Epoch 136: val_loss did not improve from 0.25815
11/11 - 0s - loss: 0.2670 - accuracy: 0.9034 - val_loss: 0.3229 - val_accuracy: 0.7727 - 39ms/epoch - 4ms/step
Epoch 137/300

Epoch 137: val_loss did not improve from 0.25815
11/11 - 0s - loss: 0.2810 - accuracy: 0.9148 - val_loss: 0.3349 - val_accuracy: 0.7727 - 41ms/epoch - 4ms/step
Epoch 138/300

Epoch 138: val_loss did not improve from 0.25815
11/11 - 0s - loss: 0.2394 - accuracy: 0.9318 - val_loss: 0.3575 - val_accuracy: 0.7727 - 37ms/epoch - 3ms/step
Epoch 139/300


Epoch 180/300

Epoch 180: val_loss did not improve from 0.23736
11/11 - 0s - loss: 0.2402 - accuracy: 0.9261 - val_loss: 0.3674 - val_accuracy: 0.7727 - 43ms/epoch - 4ms/step
Epoch 181/300

Epoch 181: val_loss did not improve from 0.23736
11/11 - 0s - loss: 0.2353 - accuracy: 0.9318 - val_loss: 0.3473 - val_accuracy: 0.7727 - 42ms/epoch - 4ms/step
Epoch 182/300

Epoch 182: val_loss did not improve from 0.23736
11/11 - 0s - loss: 0.2437 - accuracy: 0.9261 - val_loss: 0.3376 - val_accuracy: 0.7727 - 43ms/epoch - 4ms/step
Epoch 183/300

Epoch 183: val_loss did not improve from 0.23736
11/11 - 0s - loss: 0.1930 - accuracy: 0.9432 - val_loss: 0.3453 - val_accuracy: 0.7727 - 39ms/epoch - 4ms/step
Epoch 184/300

Epoch 184: val_loss did not improve from 0.23736
11/11 - 0s - loss: 0.2098 - accuracy: 0.9432 - val_loss: 0.3354 - val_accuracy: 0.7727 - 38ms/epoch - 3ms/step
Epoch 185/300

Epoch 185: val_loss did not improve from 0.23736
11/11 - 0s - loss: 0.2763 - accuracy: 0.8977 - val_loss: 0.31

11/11 - 0s - loss: 0.1951 - accuracy: 0.9432 - val_loss: 0.2316 - val_accuracy: 0.9091 - 41ms/epoch - 4ms/step
Epoch 227/300

Epoch 227: val_loss did not improve from 0.19817
11/11 - 0s - loss: 0.1944 - accuracy: 0.9545 - val_loss: 0.2307 - val_accuracy: 0.9091 - 39ms/epoch - 4ms/step
Epoch 228/300

Epoch 228: val_loss did not improve from 0.19817
11/11 - 0s - loss: 0.2003 - accuracy: 0.9545 - val_loss: 0.2368 - val_accuracy: 0.9091 - 38ms/epoch - 3ms/step
Epoch 229/300

Epoch 229: val_loss did not improve from 0.19817
11/11 - 0s - loss: 0.2062 - accuracy: 0.9375 - val_loss: 0.2206 - val_accuracy: 0.9091 - 37ms/epoch - 3ms/step
Epoch 230/300

Epoch 230: val_loss did not improve from 0.19817
11/11 - 0s - loss: 0.2075 - accuracy: 0.9432 - val_loss: 0.2139 - val_accuracy: 0.9091 - 40ms/epoch - 4ms/step
Epoch 231/300

Epoch 231: val_loss did not improve from 0.19817
11/11 - 0s - loss: 0.2070 - accuracy: 0.9261 - val_loss: 0.2100 - val_accuracy: 0.9091 - 40ms/epoch - 4ms/step
Epoch 232/300


Epoch 273/300

Epoch 273: val_loss did not improve from 0.18300
11/11 - 0s - loss: 0.1952 - accuracy: 0.9261 - val_loss: 0.1912 - val_accuracy: 0.9091 - 42ms/epoch - 4ms/step
Epoch 274/300

Epoch 274: val_loss did not improve from 0.18300
11/11 - 0s - loss: 0.1552 - accuracy: 0.9602 - val_loss: 0.2067 - val_accuracy: 0.9091 - 39ms/epoch - 4ms/step
Epoch 275/300

Epoch 275: val_loss did not improve from 0.18300
11/11 - 0s - loss: 0.1725 - accuracy: 0.9659 - val_loss: 0.2110 - val_accuracy: 0.9091 - 37ms/epoch - 3ms/step
Epoch 276/300

Epoch 276: val_loss did not improve from 0.18300
11/11 - 0s - loss: 0.1468 - accuracy: 0.9716 - val_loss: 0.1929 - val_accuracy: 0.9091 - 43ms/epoch - 4ms/step
Epoch 277/300

Epoch 277: val_loss did not improve from 0.18300
11/11 - 0s - loss: 0.1696 - accuracy: 0.9773 - val_loss: 0.1857 - val_accuracy: 0.9091 - 44ms/epoch - 4ms/step
Epoch 278/300

Epoch 278: val_loss improved from 0.18300 to 0.18088, saving model to /tmp/checkpoint.h5
11/11 - 0s - loss: 0.

In [284]:
# load optimal weights
model.load_weights('/tmp/checkpoint.h5')

In [285]:
probs       = model.predict(X_test)
preds       = probs.argmax(axis = -1)  
acc         = np.mean(preds == y_test.argmax(axis=-1))
print("Classification accuracy: %f " % (acc))

1/1 [==============================] - 0s 55ms/step
Classification accuracy: 0.909091 


In [251]:
model.save_weights('../models/EEGnet_weight_0.95')

In [252]:
model.save('../models/EEGnet_model_0.95')

INFO:tensorflow:Assets written to: ../models/EEGnet_model_0.95/assets


INFO:tensorflow:Assets written to: ../models/EEGnet_model_0.95/assets
